In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
    break

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install facenet_pytorch --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.4 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sat Feb 15 20:44:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from typing import List, Tuple, Dict, Union
from collections import defaultdict
import os


def convert_path(path: str) -> str:
    for i in range(10):
        path = path.replace(f'{i:02}', str(i))
    return path


def process_file_paths(df: pd.DataFrame) -> List[Tuple[str, str]]:
    result1: List[Tuple[str, str]] = []
    result: List[Tuple[str, str]] = []
    df_only = pd.read_csv('/kaggle/input/testdata/faces_metadata.csv')

    for _, row in df_only.iterrows():
        file_name = row['file_path']

        file_path = file_name

        result1.append((file_path, row['label']))

    groups = defaultdict(list)
    for file_path, label in result1:
        base_name = os.path.basename(file_path).split('_')[0]
        groups[base_name].append(file_path)

    single_element_groups = [name for name, group in groups.items() if len(group) == 1]

    for _, row in df.iterrows():
        file_name = row['path']
        checker = str(row['file_name']).replace('.mp4', '')
        if checker in single_element_groups:
            path_parts = file_name.split('\\')
            curr_path = path_parts[-1].replace('.zip', '')
            file_path = os.path.join('/kaggle/input/deepfake-detection', row['label'], row['label'], convert_path(curr_path), row['file_name'])
            file_path_target = os.path.join('/kaggle/working/deepfake-detection', row['label'], row['label'], convert_path(curr_path))
            file_path_target_2 = os.path.join('deepfake-detection', row['label'], row['label'], convert_path(curr_path))
            result.append((file_path, row['label'], file_path_target, row['file_name'], file_path_target_2))

    return result


df = pd.read_csv('/kaggle/input/deepfake-detection/selected_video.csv')
result = process_file_paths(df)
len(result)

4501

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
import cv2
import torch
import numpy as np
import os
from facenet_pytorch import MTCNN
from tqdm import tqdm
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
detector = MTCNN(device='cuda', post_process=False)


def process_frame(frame, previous_box):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    boxes, _ = detector.detect(frame, landmarks=False)

    if boxes is None:
        return None, previous_box

    if previous_box is not None:
        closest_box = min(boxes, key=lambda box: np.linalg.norm(np.array([(box[0] + box[2]) / 2, (box[1] + box[3]) / 2]) -
                                                             np.array([(previous_box[0] + previous_box[2]) / 2, (previous_box[1] + previous_box[3]) / 2])))
    else:
        closest_box = boxes[0]

    width = closest_box[2] - closest_box[0]
    height = closest_box[3] - closest_box[1]
    expand_x = width * 0.3 / 2
    expand_y = height * 0.3 / 2
    x1 = max(int(closest_box[0] - expand_x), 0)
    y1 = max(int(closest_box[1] - expand_y), 0)
    x2 = min(int(closest_box[2] + expand_x), frame.shape[1])
    y2 = min(int(closest_box[3] + expand_y), frame.shape[0])

    face = frame[y1:y2, x1:x2]
    face = cv2.resize(face, (600, 600))
    face = torch.from_numpy(face).permute(2, 0, 1).float() / 255.0

    return face, closest_box


def extract_frames(video_path, required_frames=20):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    desired_indices = list(np.linspace(0, total_frames - 1, num=required_frames, dtype=int))

    face_frames = []
    previous_box = None
    current_index = 0

    for target_index in desired_indices:
        while current_index < target_index:
            ret = cap.grab()
            if not ret:
                break
            current_index += 1

        ret, frame = cap.read()
        if not ret:
            break
        current_index += 1

        face, current_box = process_frame(frame, previous_box)
        if face is not None:
            face_frames.append(face)
            previous_box = current_box

    cap.release()

    return [face_frames] if len(face_frames) >= required_frames else [], fps


def save_sequences_as_videos(sequences, output_dir, name, fps, label, df_path):
    os.makedirs(output_dir, exist_ok=True)

    data = []
    for idx, sequence in enumerate(sequences):
        curr_name = name.replace('.mp4', '')
        temp_name = f'{curr_name}_{idx}.mp4'
        video_path = os.path.join(output_dir, temp_name)
        height, width = sequence[0].shape[1], sequence[0].shape[2]
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(video_path, fourcc, fps, (width, height))

        for face in sequence:
            frame = (face.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
            video_writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

        video_writer.release()
        data.append({'file_path': os.path.join(df_path, temp_name), 'label': label})
    return data


temp_data = []
for file_path, label, folder, file_name, save_path_df in tqdm(result, leave=False):
    res, fps = extract_frames(file_path, 20)
    print(file_path)
    curr = save_sequences_as_videos(res, folder, file_name, fps, label, save_path_df)
    temp_data.extend(curr)
    print(len(res))

df = pd.DataFrame(temp_data)
df.to_csv("/kaggle/working/faces_metadata.csv", index=False)


  0%|          | 1/4501 [00:04<5:24:11,  4.32s/it]

/kaggle/input/deepfake-detection/FAKE/FAKE/dfdc_train_part_9/qudhlvaxtc.mp4
1


  0%|          | 2/4501 [00:08<5:29:26,  4.39s/it]

/kaggle/input/deepfake-detection/FAKE/FAKE/dfdc_train_part_4/jzxzzmcplq.mp4
0


  0%|          | 3/4501 [00:12<4:54:52,  3.93s/it]

...

100%|█████████▉| 4499/4501 [4:08:33<00:06,  3.02s/it]

/kaggle/input/deepfake-detection/REAL/REAL/dfdc_train_part_9/icffpvvtnu.mp4
1


100%|█████████▉| 4500/4501 [4:08:35<00:02,  2.98s/it]

/kaggle/input/deepfake-detection/REAL/REAL/dfdc_train_part_9/ucidfsqdkw.mp4
1


/kaggle/input/deepfake-detection/REAL/REAL/dfdc_train_part_9/mhpavogfxb.mp4
1


In [ ]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = YOUR_KAGGLE_API_TOKEN

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
import json


metadata = {
    "title": "stepdata20",
    "id": "vasylhunia/stepdata20",
    "licenses": [{"name": "CC0-1.0"}]
}

file_path = "/kaggle/working/dataset-metadata.json"

with open(file_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"Файл dataset-metadata.json створено за шляхом: {file_path}")


Файл dataset-metadata.json створено за шляхом: /kaggle/working/dataset-metadata.json


In [ ]:
!kaggle datasets create -p /kaggle/working --dir-mode zip

Starting upload for file __notebook__.ipynb
100%|██████████████████████████████████████| 1.57M/1.57M [00:00<00:00, 3.75MB/s]
Upload successful: __notebook__.ipynb (2MB)
Starting upload for file faces_metadata.csv
100%|█████████████████████████████████████████| 291k/291k [00:00<00:00, 729kB/s]
Upload successful: faces_metadata.csv (291KB)
Starting upload for file deepfake-detection.zip
100%|█████████████████████████████████████████| 863M/863M [00:06<00:00, 146MB/s]
Upload successful: deepfake-detection.zip (863MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/vasylhunia/stepdata20
